In [ ]:
from bs4 import BeautifulSoup
import requests
import csv
# from requests_html import HTMLSession

In [ ]:
entries = []
entry = []
urlnumber = 200 # Give the page number to start with


while urlnumber < 310: # Give the page number to end with
    print(urlnumber)
    url = 'http://forums.edmunds.com/discussion/2864/general/x/entry-level-luxury-performance-sedans/p%d' % (urlnumber,) # Give the url of the forum, excluding the page number in the hyperlink

    try:
        r = requests.get(url, timeout = 10) # Sending a request to access the page
    except Exception as e:
        print("Error message:",e)
        break;

    data = r.text
    
    soup = BeautifulSoup(data, 'lxml') # Getting the page source into the soup
    
    for div in soup.find_all('div'):
        entry = []
        if(div.get('class') != None and div.get('class')[0] == 'Comment'): # A single post is referred to as a comment. Each comment is a block denoted in a div tag which has a class called comment.
            ps = div.find_all('p') # gets all the tags called p to a variable ps
            aas = div.find_all('a') # gets all the tags called a to a variable aas
            spans = div.find_all('span')
            times = div.find_all('time') # used to extract the time tag which gives the iDate of the post

            concat_str = ''
            for str in aas[1].contents: # prints the contents that is between the tag start and end
                if str != "<br>" or str != "<br/>": # breaks in post which we need to work around
                    concat_str = (concat_str + ' '+ str).encode("utf-8").strip() # the format extracted is a unicode - we need a uniform structure to work with the strings
            entry.append(concat_str)

            concat_str = ''
            for str in times[0].contents:
                if str != "<br>" or str != "<br/>":
                    concat_str = (concat_str + ' '+ str).encode('iso-8859-1').strip()
            entry.append(concat_str)

            for div in div.find_all('div'):
                if (div.get('class') != None and div.get('class')[0] == 'Message'): # extracting the div tag with the class attribute as message
                    blockquotes = []
                    x = div.get_text()
                    for bl in div.find_all('blockquote'):
                        blockquotes.append(bl.get_text()) # block quote is used to get the quote made by a person. get_text helps to eliminate the hyperlinks and pulls out only the data.
                        bl.decompose()
                    # Encoding the text to ascii code by replacing the non-ascii characters
                    ascii_encoding = div.get_text().replace("\n"," ").replace("<br/>","").encode('ascii','replace')
                    # Convert the ASCII encoding to Latin1 encoding
                    latin1_encoding = ascii_encoding.decode('ascii').encode('iso-8859-1')
                    # Append the encoding bytes to output list
                    entry.append(latin1_encoding)

                    for bl in blockquotes:
                        ascii_encoding = bl.replace("\n"," ").replace("<br/>","").encode('ascii','replace')
                        latin1_encoding = ascii_encoding.decode('ascii').encode('iso-8859-1')
                        entry.append(latin1_encoding)

            entries.append(entry)
            
    urlnumber += 1

# Convert a list of byte to list a of string     
stringlist=[[x.decode('iso-8859-1') for x in entry] for entry in entries]
# Save the list to a csv file
with open('edmunds_extraction1.csv', 'w') as output:
    writer = csv.writer(output, quoting=csv.QUOTE_ALL)
    writer.writerows(stringlist)

print ("Wrote to edmunds_extraction1.csv")

200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
277
278
279
280
281
282
283
284
285
286
287
288
289
290
291
292
293
294
295
296
297
298
299
300
301
302
303
304
305
306
307
308
309
Wrote to edmunds_extraction1.csv


In [ ]:
import pandas as pd

review = pd.read_csv('edmunds_extraction1.csv',names=['User','Date','Review'],header=None)
                     #,parse_dates=['Date'])

In [ ]:
review

,User,Date,Review
0,habitat1,September 2007,The car doesn't appeal to me for a number of ...
1,kdshapiro,September 2007,"""I think without the ""buy American"" loyalty,""..."
2,fedlawman,September 2007,"""It is a matter of putting out a seriously co..."
3,designman,September 2007,I expect to see this label soon:Outsourced in...
4,markcincinnati,September 2007,"I do like the CTS, I will certainly consider ..."
...,...,...,...
5495,ivan_99,April 2013,"Since your passengers are kids, not adults, l..."
5496,dino001,April 2013,"Yeah, it does look nice on pictures. Deep sea..."
5497,dino001,April 2013,"Did not think of that, you make sense. So it ..."
5498,dino001,April 2013,Didn't Accord just get smaller in this genera...


In [ ]:
mdl_list = pd.read_csv('models.csv',names=['Brand','Model'],header=None)

In [ ]:
mdl_list 

,Brand,Model
0,acura,integra
1,acura,Legend
2,acura,vigor
3,acura,rlx
4,acura,ILX
...,...,...
523,volvo,xc90
524,volvo,s60
525,volvo,s80
526,volvo,v60


In [ ]:
import nltk
nltk.download('punkt')
from nltk.tokenize import sent_tokenize, word_tokenize

[nltk_data] Downloading package punkt to /Users/ziyezhang/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [ ]:
tokenized = []
for idx in range(len(review)):
    sentence = word_tokenize(review.loc[idx,"Review"])
    for i in range(len(sentence)):
        if any(sentence[i]==mdl_list['Model']):
            print(sentence[i])
            brand = mdl_list.loc[mdl_list.Model==sentence[i],'Brand'].iloc[0]
            sentence[i] = brand
    tokenized.append(sentence)


Corvette
cars
cars
cars
sedans
cars
cars
Camaro
Camaro
cars
cars
cars
Camaro
cars
cars
cars
cars
Pilot
335i
335i
seats
seats
problems
3-series
3-series
dart
cars
3-series
5-series
535i
cars
cars
535i
3-series
cars
cars
535i
535i
problems
problems
cars
cars
3-series
problems
problems
M3
cars
3-series
cars
M3
caliber
caliber
caliber
Camry
cars
5-series
528i
M3
3-series
5-series
335i
cars
cars
cars
seats
problems
cars
MDX
cars
cars
cars
cars
cars
cars
cars
cars
cars
cars
cars
cars
cars
3-series
3-series
3-series
cars
cars
335i
sedans
sedans
sedans
328i
335i
335i
seats
335i
seats
335i
335i
335i
335i
335i
335i
335i
Shuttle
cars
335i
335i
cars
cars
335i
3-series
128i
335i
128i
328i
128i
Comfort
seats
seats
335i
335i
cars
cars
cars
328i
328i
cars
M5
Tahoe
335i
3-series
335i
M3
535i
535i
535i
3-series
3-series
335i
3-series
328i
3-series
3-series
335i
335i
335i
M5
335i
328i
128i
Click
cars
535i
3-series
5-series
cars
535i
3-series
5-series
M5
M5
335i
M5
M5
M5
cars
335i
335i
335i
M5
M5
M5
M5
M5

cars
cars
cars
NSX
cars
cars
cars
TSX
cars
cars
328i
328i
335i
cars
cars
seats
cars
335i
xdrive
seats
cars
335i
328i
cars
335i
cars
cars
328i
cars
cars
cars
cars
cars
cars
cars
cars
cars
cars
cars
RX-8
RX-8
328i
sedans
RX-8
RX-8
RX-8
cars
TSX
cars
cars
cars
cars
seats
Tahoe
cars
335i
335i
335i
cars
cars
128i
128i
TSX
TSX
M3
cars
TSX
TSX
Corvette
Legend
Outback
RX-8
328i
cars
cars
cars
cars
cars
cars
quattro
seats
cars
cars
335i
cars
cars
TSX
TSX
TSX
cars
accord
Civic
TSX
TSX
TSX
sedans
3-series
problems
TSX
RSX
335i
cars
335i
seats
cars
seats
TSX
TSX
seats
TSX
Prelude
TSX
Pilot
cars
seats
5er
M3
M3
charger
M3
M3
M3
Focus
Cruze
cars
cars
M3
Pilot
M3
a4
vw
vw
sedans
3-series
Pilot
M3
M3
M5
seats
cars
cars
cars
TSX
a4
cars
accord
crv
accords
hondas
TSX
TSX
cars
accord
accord
TSX
TSX
sedans
TSX
cars
cars
TSX
cars
cars
cars
TSX
accord
jetta
TSX
TSX
jetta
cars
cars
cars
cars
cars
TSX
cars
TSX
TSX
TSX
TSX
TSX
jetta
cars
TSX
TSX
TSX
TSX
TSX
TSX
accord
cars
cars
accord
TSX
TSX
seats
cars
cars
T

328i
M3
3er
cars
335i
335i
cars
cars
335i
328i
335i
335i
M3
335i
M3
335i
328i
cars
335i
mustang
335i
335i
335i
Pilot
Civic
Focus
Camaro
cars
Prelude
Prelude
TSX
Prelude
328i
Focus
Camry
cars
cars
cars
mustang
cars
535i
seats
bmws
g37
cars
cars
cars
seats
Camaro
TSX
accord
cars
cars
Genesis
TSX
seats
cars
cars
TSX
cars
cars
cars
cars
cars
cars
cars
328i
cars
cars
cars
Camry
Civic
versa
3-series
cars
Camry
Civic
sedans
Camry
3-series
Equus
Equus
Genesis
cars
problems
problems
problems
problems
Legacy
benz
Equus
cars
Genesis
cars
Tahoe
328i
cars
Prelude
cars
corvette
cars
Corvette
cars
cars
cars
cars
cars
cars
cars
328i
Civic
cars
problems
cars
cars
problems
cars
cars
RSX
cars
cars
Camaro
passat
cars
cars
cars
cars
cars
cars
cars
cars
cars
century
cars
cars
cars
cars
cars
cars
cars
TSX
spark
cars
cars
cars
335i
problems
335i
328i
Century
cars
Pilot
xdrive
xdrive
3er
seats
TSX
cars
cars
seats
seats
Genesis
seats
vibe
seats
problems
335i
cars
cars
cars
cars
Prius
3er
cars
cars
vw
vw
335d
IL

cars
328i
quattro
quattro
cars
cars
cars
M3
M3
cars
cars
cars
Camaro
Camaro
cars
cars
328i
328i
5-series
seats
seats
seats
3-series
seats
seats
seats
3-series
seats
seats
seats
seats
seats
cars
cars
quattro
quattro
cars
328i
problems
cars
cars
volt
5-series
seats
Camry
seats
cars
problems
335i
seats
seats
seats
5-series
3-series
cars
RDX
seats
accord
NSX
NSX
NSX
NSX
NSX
NSX
NSX
NSX
NSX
golf
volt
NSX
NSX
NSX
NSX
NSX
NSX
NSX
NSX
cars
NSX
NSX
NSX
NSX
NSX
M3
NSX
NSX
NSX
NSX
NSX
NSX
problems
accord
accord
accord
cars
sedans
problems
cars
ILX
RSX
TSX
Civic
535i
accord
accord
leaf
accord
accord
cars
RSX
TSX
cars
cars
golf
cars
charger
Prelude
M3
MDX
cars
Stream
cars
cars
Express
cars
cars
golf
M3
M3
MDX
Camry
Camry
cars
335i
328i
cars
cars
Camry
M3
M5
M6
328i
328i
bmws
cars
RDX
Echo
Camry
Camry
Camry
Camry
cars
cars
Camry
Camry
cars
cars
cars
Camry
Camry
Camry
Camry
Camry
Focus
cars
cars
Camry
cars
cars
Prius
City
cars
Camry
Camry
Camry
Camry
cars
cars
Camry
335i
M3
M3
M3
seats
335i
M3
M3
M3


In [ ]:
len(tokenized) == len(review)

True

In [ ]:
review['brand_review']=tokenized